In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 1. Corpus and tokenizer
corpus = ["hello how are you", "how are you doing", "are you going out", "how is your health"]
tok = Tokenizer(); tok.fit_on_texts(corpus)

# 2. Create input-output sequences
seq = []
for line in corpus:
    tokens = tok.texts_to_sequences([line])[0]
    for i in range(1, len(tokens)):
        seq.append(tokens[:i+1])
max_len = max(len(s) for s in seq)
seq = pad_sequences(seq, maxlen=max_len, padding='pre')
x, y = seq[:, :-1], to_categorical(seq[:, -1], num_classes=len(tok.word_index)+1)

# 3. LSTM model
model = Sequential([
    Embedding(len(tok.word_index)+1, 10, input_length=max_len-1),
    LSTM(50),
    Dense(len(tok.word_index)+1, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y, epochs=300, verbose=0)
print("✅ Training complete")

# 4. Prediction
def predict(text):
    t = tok.texts_to_sequences([text])[0]
    t = pad_sequences([t], maxlen=max_len-1, padding='pre')
    p = np.argmax(model.predict(t, verbose=0))
    for word, i in tok.word_index.items():
        if i == p:
            return word
    return "?"

# 5. Run loop
while True:
    inp = input("Enter text: ")
    if inp.lower() == "exit":
        break
    print("Next word:", predict(inp))


✅ Training complete
